In [24]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time as t
import urllib

# تغيير الدليل
os.chdir(r"D:\Profile\Downloads\Ancient Egyption monuments")

# إنشاء مجلد للتنزيل
name = "Pharaoh Mummies"
os.makedirs(f"dataset_download/{name}", exist_ok=True)

# إعداد متصفح Chrome
chrome_service = Service('chromedriver.exe')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])

driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# فتح الصفحة
strr = "https://images.search.yahoo.com/search/images;_ylt=AwrihTPRzx9nqfQNZRyJzbkF;_ylu=c2VjA3NlYXJjaARzbGsDYnV0dG9u;_ylc=X1MDOTYwNjI4NTcEX3IDMgRmcgNtY2FmZWUEZnIyA3A6cyx2OmksbTpzYi10b3AEZ3ByaWQDQS5XQWZMUlhTNENCVjVLeHVQdDU2QQRuX3JzbHQDMARuX3N1Z2cDMARvcmlnaW4DaW1hZ2VzLnNlYXJjaC55YWhvby5jb20EcG9zAzAEcHFzdHIDBHBxc3RybAMwBHFzdHJsAzE4BHF1ZXJ5A1BoYXJhb2glMjAuJTIwTXVtbWllcyUyMAR0X3N0bXADMTczMDEzODI2MA--?p=Pharaoh+.+Mummies+&fr=mcafee&fr2=p%3As%2Cv%3Ai%2Cm%3Asb-top&ei=UTF-8&x=wrt&type=E210US91105G0"
driver.get(strr)

# الانتظار حتى تحميل الصور
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//img[contains(@src, "https://")]'))  # استخدام كود يشير إلى جميع الصور
    )
except Exception as e:
    print(f"Error: {e}")
    driver.quit()

links = []
x = 1
last_height = 0

def download_image(url, filename):
    try:
        resource = urllib.request.urlopen(url)
        with open(filename, "wb") as output:
            output.write(resource.read())
    except Exception as e:
        print(f"Error downloading {url}: {e}")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    t.sleep(4)

    # إعادة تعيين pics في كل مرة
    pics = driver.find_elements(By.XPATH, '//img[contains(@src, "https://")]')
    
    for pic in pics:
        try:
            img_link = pic.get_attribute('src') 
            if img_link and img_link not in links:
                links.append(img_link)
                
                file_name = f'dataset_download/{name}/{x}.jpg'
                download_image(img_link, file_name)
                x += 1
        except Exception as e:
            print(f"Error processing image: {e}")

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

driver.close()
